# Importação e preparação dos dados

In [1]:
import pandas as pd

df = pd.read_csv('../dados/acordaos-unicos-filtrados-6000.csv', sep = '|')
df['filtrado_6000']=df['filtrado_6000'].astype(str)
df.head()

,acordao,areas,filtrado_6000
0,297/2016-P,Responsabilidade,tribunal conta união tc grupo classe plenário ...
1,366/2016-P,Finanças Públicas,tribunal conta união tc grupo classe ii plenár...
2,944/2016-P,Responsabilidade,tribunal conta união tc grupo classe plenário ...
3,30/2016-P,Direito Processual,tribunal conta união tc grupo classe plenário ...
4,55/2016-P,Pessoal,wania lucia pasquarelli nascimentotcuwania luc...


In [2]:
df.shape

(9739, 3)

In [3]:
from sklearn.preprocessing import LabelBinarizer

areas = df.groupby(['areas']).groups.keys()
lbArea = LabelBinarizer()
lbArea.fit([x for x in areas])
lbArea.classes_

array(['Competência do TCU', 'Contrato Administrativo', 'Convênio',
       'Desestatização', 'Direito Processual', 'Finanças Públicas',
       'Gestão Administrativa', 'Licitação', 'Pessoal',
       'Responsabilidade'], dtype='<U23')

In [4]:
y = lbArea.transform(df['areas'])
y.shape

(9739, 10)

In [5]:
from keras.preprocessing.text import Tokenizer
import numpy as np

vocabulario = 100000
limite_texto = 6000
dim_vetor = 50

tokenizer = Tokenizer(num_words=vocabulario)
tokenizer.fit_on_texts(df['filtrado_6000'])

sequences = tokenizer.texts_to_sequences(df['filtrado_6000'])

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Using TensorFlow backend.


Found 95423 unique tokens.


In [6]:
from keras.preprocessing.sequence import pad_sequences

x = pad_sequences(sequences, maxlen=limite_texto)

print('Shape of data tensor:', x.shape)

Shape of data tensor: (9739, 6000)


In [7]:
x.shape, y.shape

((9739, 6000), (9739, 10))

In [8]:
from gensim.models import KeyedVectors

model = KeyedVectors.load_word2vec_format('../externos/model-50.txt')

/home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [9]:
# create a weight matrix for words in training docs

embedding_matrix = np.zeros((vocabulario, dim_vetor))

ok = 0
for word, i in tokenizer.word_index.items():
    if word in model:
        embedding_matrix[i] = model[word]
        ok += 1
print('Vocabulario:', i)
print('Encontrados no modelo:', ok, '=', ok * 100. / i)

Vocabulario: 95423
Encontrados no modelo: 55676 = 58.34652023097157


# Treinamento

In [11]:
from keras.models import Sequential
from keras.layers import SimpleRNN, GRU, Dense, Embedding
from keras.optimizers import RMSprop

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(256, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 6000, 50)          5000000   
_________________________________________________________________
gru_1 (GRU)                  (None, 256)               235776    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                2570      
Total params: 5,238,346
Trainable params: 5,238,346
Non-trainable params: 0
_________________________________________________________________
Train on 7791 samples, validate on 1948 samples
Epoch 1/20
7791/7791 [==============================] - 1538s 197ms/step - loss: 1.7973 - categorical_accuracy: 0.3872 - val_loss: 1.8898 - val_categorical_accuracy: 0.2295
Epoch 2/20
7791/7791 [==============================] - 1538s 197ms/step - loss: 1.7488 - categorical_accuracy: 0.4011 - val_loss: 2.0182 - val_categorical_accuracy: 0.2223
Ep

KeyboardInterrupt: 

In [12]:
from keras.models import Sequential
from keras.layers import SimpleRNN, GRU, Dense, Embedding
from keras.optimizers import RMSprop

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=False))
model.add(GRU(256, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 6000, 50)          5000000   
_________________________________________________________________
gru_2 (GRU)                  (None, 256)               235776    
_________________________________________________________________
dense_3 (Dense)              (None, 10)                2570      
Total params: 5,238,346
Trainable params: 238,346
Non-trainable params: 5,000,000
_________________________________________________________________
Train on 7791 samples, validate on 1948 samples
Epoch 1/20
7791/7791 [==============================] - 1282s 165ms/step - loss: 1.7982 - categorical_accuracy: 0.3831 - val_loss: 1.9582 - val_categorical_accuracy: 0.2177
Epoch 2/20
7791/7791 [==============================] - 1279s 164ms/step - loss: 1.7536 - categorical_accuracy: 0.3984 - val_loss: 1.9861 - val_categorical_accuracy: 0.2

In [13]:
from keras.models import Sequential
from keras.layers import SimpleRNN, GRU, Dense, Embedding
from keras.optimizers import RMSprop

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(SimpleRNN(256, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()
history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 6000, 50)          5000000   
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 256)               78592     
_________________________________________________________________
dense_4 (Dense)              (None, 10)                2570      
Total params: 5,081,162
Trainable params: 5,081,162
Non-trainable params: 0
_________________________________________________________________
Train on 7791 samples, validate on 1948 samples
Epoch 1/20
7791/7791 [==============================] - 658s 84ms/step - loss: 1.8921 - categorical_accuracy: 0.3439 - val_loss: 1.9409 - val_categorical_accuracy: 0.2556
Epoch 2/20
7791/7791 [==============================] - 659s 85ms/step - loss: 1.8065 - categorical_accuracy: 0.3702 - val_loss: 2.0232 - val_categorical_accuracy: 0.2161
Epoch 